Выполнить Стэкинг, Бэгинг, Вотинг и Бустинг. При реализации алгоритмов не использовать готовые решения. 
За сровнение взять CatBoostClassifier, как базову для сравнения метрик качества. Сравнить результат с реализацией своих ансамблей. 
Для однозначности и интерпретируемости результатов использовать приложенный набор данных. 
При реализации бустинга - просто сокращайте набор данных на котором модель отработала хорошо (правильно предсказанные данные). 

In [2]:
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.base import clone
from sklearn.svm import SVC

from imblearn.over_sampling import ADASYN
from catboost import CatBoostClassifier, Pool
from random import sample

import pandas as pd
import numpy as np


In [3]:
# read DataFrame from csv
excel_file = 'winequality-white.csv'

wine_df = pd.read_csv(excel_file, delimiter=';')
wine_df


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
wine_df.describe()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [4]:
# check columns types and nan-values
wine_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [4]:
# set target and features columns
TARGET = 'quality'
features = wine_df.columns.tolist()
features.remove(TARGET)
print(features)


['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']


In [4]:
# check target data destribution in training set
wine_df[TARGET].value_counts()


quality
6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: count, dtype: int64

Проверили сбалансированность классов, для двух классов очень мало данных, с помощью ADASYN увеличиваем кол-во примеров в наборе данных. Затем проводим нормализацию данных и разбиваем на тренировочную и тестовую выборки.

In [5]:
# increase dataset size
adasyn = ADASYN(n_neighbors=3)
x_oversampled, y_oversamled = adasyn.fit_resample(wine_df[features], wine_df[TARGET])
y_oversamled.value_counts()


quality
7    2390
5    2224
3    2203
9    2200
6    2198
8    2195
4    2164
Name: count, dtype: int64

In [6]:
# normalize features values
scaler = MinMaxScaler(feature_range=(0, 1))
x_oversampled = scaler.fit_transform(x_oversampled)


In [7]:
# split data into train and test
train_x, test_x, train_y, test_y = train_test_split(x_oversampled, y_oversamled, train_size=0.8)
train_x.shape, test_x.shape


((12459, 11), (3115, 11))

Стэкинг

In [8]:
# realize stacking models class
class ModelStacking():
    def __init__(self, estimators, final_estimator, cv=5):
        self.estimators = estimators
        self.final_estimator = final_estimator
        self.cv = cv

    def fit(self, x, y):
        self.n_classes = len(np.unique(y))
        res_matrix = np.zeros((x.shape[0], self.n_classes * len(self.estimators)))

        for i, est in enumerate(self.estimators):
            res_matrix[:, i * self.n_classes:(i + 1) * self.n_classes] = cross_val_predict(est, x, y, cv=self.cv,
                                                                                           method='predict_proba')
            est = est.fit(x, y)

        self.final_estimator = self.final_estimator.fit(res_matrix, y)

        return self

    def predict_proba(self, x):
        res_matrix = np.zeros((x.shape[0], self.n_classes * len(self.estimators)))

        for i, est in enumerate(self.estimators):
            res_matrix[:, i * self.n_classes:(i + 1) * self.n_classes] = est.predict_proba(x)

        proba_y = self.final_estimator.predict_proba(res_matrix)
        return proba_y

    def predict(self, x):
        proba_y = self.predict_proba(x)
        return self.final_estimator.classes_[np.argmax(proba_y, axis=1)]



In [9]:
# fit and predict stacking model
cv = 5
model_stacking = ModelStacking(estimators=[GaussianNB(), SVC(probability=True), KNeighborsClassifier()], 
                               final_estimator=DecisionTreeClassifier(), cv=cv)
model_stacking = model_stacking.fit(train_x, train_y)

stack_pred_proba = model_stacking.predict_proba(test_x)
stack_pred_y = model_stacking.predict(test_x)


In [10]:
stack_score = accuracy_score(test_y, stack_pred_y)
stack_score


0.771107544141252

In [66]:
res_df = pd.DataFrame(dict(zip(['Ensamble', 'Accuracy'], ['Stacking', stack_score])), index=[0])
res_df


,Ensamble,Accuracy
0,Stacking,0.771108


Бэгинг

In [15]:
# realize bagging models class
class ModelBagging():
    def __init__(self, estimator, n_estimators):
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.estimators = list()

    def fit(self, x, y):
        idxs = range(len(x))

        for _ in range(self.n_estimators):
            curr_idxs = sample(idxs, int(0.632 * len(idxs)))
            est = clone(self.estimator)
            est = est.fit(x[curr_idxs], y.iloc[curr_idxs])
            self.estimators.append(est)

        return self

    def predict(self, x):
        preds = None

        for est in self.estimators:
            curr_pred = est.predict(x)

            if preds is None:
                preds = curr_pred
            else:
                preds = np.vstack((preds, curr_pred))

        return np.median(preds, axis=0).astype(int)


In [16]:
# fit and predict bagging model
model_bagging = ModelBagging(estimator=DecisionTreeClassifier(), n_estimators=150)
model_bagging = model_bagging.fit(train_x, train_y)
bag_pred_y = model_bagging.predict(test_x)


In [17]:
bag_score = accuracy_score(test_y, bag_pred_y)
bag_score


0.8735152487961477

In [67]:
res_df = res_df._append(dict(zip(res_df.columns, ['Bagging', bag_score])), ignore_index=True)
res_df


,Ensamble,Accuracy
0,Stacking,0.771108
1,Bagging,0.873515


Вотинг

In [19]:
# realize voting models class
class ModelVoting():
    def __init__(self, estimators):
        self.estimators = estimators

    def fit(self, x, y):
        for est in self.estimators:
            est = est.fit(x, y)

        return self

    def predict_hard(self, x):
        preds = None

        for est in self.estimators:
            curr_pred = est.predict(x)

            if preds is None:
                preds = curr_pred
            else:
                preds = np.vstack((preds, curr_pred))

        return np.median(preds, axis=0).astype(int)

    def predict_soft(self, x):
        probs = np.zeros((x.shape[0], len(self.estimators[0].classes_)))

        for est in self.estimators:
            probs += est.predict_proba(x)

        return self.estimators[0].classes_[np.argmax(probs, axis=1)]


In [20]:
# fit voting model and predict hard and soft methods
model_voting = ModelVoting(estimators=[GaussianNB(), SVC(probability=True), KNeighborsClassifier(), DecisionTreeClassifier()])
model_voting = model_voting.fit(train_x, train_y)
vot_pred_hard = model_voting.predict_hard(test_x)
vot_pred_soft = model_voting.predict_soft(test_x)


In [21]:
vot_score_hard = accuracy_score(test_y, vot_pred_hard)
vot_score_hard


0.7460674157303371

In [22]:
vot_score_soft = accuracy_score(test_y, vot_pred_soft)
vot_score_soft


0.8362760834670947

In [68]:
res_df = pd.concat([res_df, pd.DataFrame(dict(zip(res_df.columns, [['Voting hard', 'Voting soft'], [vot_score_hard, vot_score_soft]])))],
                    ignore_index=True, axis=0)
res_df


,Ensamble,Accuracy
0,Stacking,0.771108
1,Bagging,0.873515
2,Voting hard,0.746067
3,Voting soft,0.836276


Бустинг

In [42]:
# realize boosting models class
class ModelBoosting():
    def __init__(self, estimator, n_estimators):
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.estimators = list()

    def fit(self, x, y):
        x_train = x.copy()
        y_train = y.copy()

        self.n_classes = len(np.unique(y))
        n_samples = len(y)

        for _ in range(self.n_estimators):
            est = clone(self.estimator)
            est = est.fit(x_train, y_train)
            preds = est.predict(x)

            errs = preds != y
            x_train = x_train[errs].copy()
            y_train = y_train[errs].copy()

            self.estimators.append(est)

            if len(y_train) < 0.3 * n_samples:
                break

        return self

    def predict(self, x):
        probs = np.zeros((x.shape[0], len(self.estimators[0].classes_)))

        for est in self.estimators:
            probs += est.predict_proba(x)

        return self.estimators[0].classes_[np.argmax(probs, axis=1)]


In [49]:
# fit and predict boosting model
model_boosting = ModelBoosting(estimator=DecisionTreeClassifier(), n_estimators=50)
model_boosting = model_boosting.fit(train_x, train_y)
boost_pred_y = model_boosting.predict(test_x)


In [50]:
boost_score = accuracy_score(test_y, boost_pred_y)
boost_score


0.8256821829855537

In [69]:
res_df = res_df._append(dict(zip(res_df.columns, ['Boosting', boost_score])), ignore_index=True)
res_df


,Ensamble,Accuracy
0,Stacking,0.771108
1,Bagging,0.873515
2,Voting hard,0.746067
3,Voting soft,0.836276
4,Boosting,0.825682


CatBoostClassifier

In [8]:
train_pool = Pool(train_x, train_y)
test_pool = Pool(test_x, test_y)


In [ ]:
catboost_model = CatBoostClassifier()
catboost_model = catboost_model.fit(train_pool)
cat_pred_y = catboost_model.predict(test_pool)


In [11]:
cat_score = accuracy_score(test_y, cat_pred_y)
cat_score


0.8876404494382022

In [12]:
res_df = res_df._append(dict(zip(res_df.columns, ['CatBoostClassifier', cat_score])), ignore_index=True)
res_df


,Ensamble,Accuracy
0,Stacking,0.771108
1,Bagging,0.873515
2,Voting hard,0.746067
3,Voting soft,0.836276
4,Boosting,0.825682
5,CatBoostClassifier,0.887640


In [18]:
res_df = res_df.sort_values(by='Accuracy')
res_df


,Ensamble,Accuracy
2,Voting hard,0.746067
0,Stacking,0.771108
4,Boosting,0.825682
3,Voting soft,0.836276
1,Bagging,0.873515
5,CatBoostClassifier,0.887640


Как видно из таблицы выше, никакой из написанных методов не дал точности выше CatBoostClassifier. Лучше всего отработал Бэггинг моделей.